In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model


In [ ]:
pip install tf-models-official


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.8 MB/s eta 0:00:00
  Created wheel for seqeval:

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the SSD MobileNet V2 pre-trained model from TensorFlow Hub
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"
model = hub.load(model_url)

# Print model signature to verify successful loading
print(model.signatures.keys())


KeysView(_SignatureMap({'serving_default': <ConcreteFunction (*, input_tensor: TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name='input_tensor')) -> Dict[['raw_detection_boxes', TensorSpec(shape=(1, 1917, 4), dtype=tf.float32, name='raw_detection_boxes')], ['detection_multiclass_scores', TensorSpec(shape=(1, 100, 91), dtype=tf.float32, name='detection_multiclass_scores')], ['detection_classes', TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_classes')], ['detection_boxes', TensorSpec(shape=(1, 100, 4), dtype=tf.float32, name='detection_boxes')], ['raw_detection_scores', TensorSpec(shape=(1, 1917, 91), dtype=tf.float32, name='raw_detection_scores')], ['num_detections', TensorSpec(shape=(1,), dtype=tf.float32, name='num_detections')], ['detection_anchor_indices', TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_anchor_indices')], ['detection_scores', TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_scores')]] at 0x7AFA237DD1E0>}))


In [ ]:
def detect_objects(frame):
    # Preprocess frame for SSD MobileNet V2 model
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run model inference
    detections = model(input_tensor)

    # Extract and decode relevant information
    bboxes = detections["detection_boxes"][0].numpy()  # bounding boxes
    class_ids = detections["detection_classes"][0].numpy().astype(int)  # class IDs
    scores = detections["detection_scores"][0].numpy()  # confidence scores

    return bboxes, class_ids, scores


In [ ]:
pip install requests


In [ ]:
import requests


In [ ]:
# Load COCO labels
labels_url = "https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt"
labels = requests.get(labels_url).text.splitlines()


In [ ]:
import cv2

# Load COCO labels
labels_url = "https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt"
labels = requests.get(labels_url).text.splitlines()

# Open webcam
cap = cv2.VideoCapture(0)
CONFIDENCE_THRESHOLD = 0.8

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection on current frame
    bboxes, class_ids, scores = detect_objects(frame)

    # Draw boxes and labels
    height, width, _ = frame.shape
    for bbox, class_id, score in zip(bboxes, class_ids, scores):
        if score > CONFIDENCE_THRESHOLD:
            ymin, xmin, ymax, xmax = bbox
            left, top, right, bottom = int(xmin * width), int(ymin * height), int(xmax * width), int(ymax * height)
            label = f"{labels[class_id]}: {score:.2f}"

            # Draw bounding box and label
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the frame with detections
    cv2.imshow("SSD Real-time Object Detection", frame)

    # Exit loop on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Function to run object detection
def detect_objects(frame):
    input_tensor = tf.convert_to_tensor(np.array(frame))
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = model(input_tensor)

    bboxes = detections["detection_boxes"][0].numpy()  # Bounding boxes
    class_ids = detections["detection_classes"][0].numpy().astype(int)  # Class IDs
    scores = detections["detection_scores"][0].numpy()  # Confidence scores

    return bboxes, class_ids, scores


In [ ]:
def run_real_time_detection():
    while True:
        # Capture frame from webcam
        img = capture_image()
        frame = np.array(img)

        # Run detection
        bboxes, class_ids, scores = detect_objects(frame)

        # Draw boxes and labels on the frame
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(frame)

        height, width, _ = frame.shape
        for bbox, class_id, score in zip(bboxes, class_ids, scores):
            if score > CONFIDENCE_THRESHOLD:
                ymin, xmin, ymax, xmax = bbox
                left, top, right, bottom = int(xmin * width), int(ymin * height), int(xmax * width), int(ymax * height)

                # Adjust label index and ensure it's within bounds
                adjusted_class_id = class_id - 1
                if 0 <= adjusted_class_id < len(labels):
                    label = f"{labels[adjusted_class_id]}: {score:.2f}"
                else:
                    label = f"Unknown: {score:.2f}"

                # Draw bounding box
                rect = plt.Rectangle((left, top), right - left, bottom - top, fill=False, color="red", linewidth=2)
                ax.add_patch(rect)

                # Draw label
                plt.text(left, max(top - 10, 0), label, color="white", fontsize=12,
                         bbox=dict(facecolor="red", alpha=0.5))

        # Show the frame with detections
        plt.axis("off")
        display(fig)
        plt.close(fig)


In [ ]:
run_real_time_detection()
